In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
df = pd.read_json('../../data/train/trainText-75-25.json', lines=True)

In [2]:
ingredients = {}
for lst in df['ingredientsFiltered']:
    for i in lst:
        i = i.replace(' ','_').strip('_')
        if not i in ingredients:
            ingredients[i] = 1
        else:
            ingredients[i] += 1

In [3]:
ingredients['black_olives']

774

In [4]:
MIN_FREQUENCY = 10
filtered_ingredients = {k: v for k, v in ingredients.items() if v >= MIN_FREQUENCY}

In [5]:
recipes = list(df['recipe'])
labels = list(df['cuisine'])

In [6]:
vectorizer = CountVectorizer()
vectorizer.fit(filtered_ingredients.keys())
print('Number of Features: %d'%len(vectorizer.get_feature_names()))

Number of Features: 3092


In [7]:
print(vectorizer.get_feature_names())

[u'____oz', u'_alfredo_sauce', u'_asiago', u'_beaten', u'_best_food_mayonnais', u'_best_food_real_mayonnais', u'_butter', u'_capicola', u'_chicken_breast', u'_chicken_broth', u'_chicken_tenderloins', u'_chinese_black_vinegar_dark_soy_sauce_chicken_stock_peanut_oil_red_onion', u'_chop', u'_chunky_mild_salsa', u'_classic_hummus', u'_classico_olive_oil', u'_cook_drain', u'_cookies', u'_crush', u'_crushed_finely_grated_ginger_\xbd_sichuan_peppercorns', u'_crushed_long_dried_red_chillies', u'_dash', u'_diced', u'_diced_tomatoes', u'_drain', u'_eggs', u'_fiesta_sides_spanish_rice', u'_finely', u'_garlic_cloves', u'_genoa_salami', u'_green_chiles', u'_half', u'_half_lengthways', u'_halved_thinly_lengthways_', u'_italian_loaf', u'_less', u'_lightly_beaten', u'_manchego', u'_meats_fish', u'_natural_sweetener', u'_parmigiano', u'_peel_devein', u'_pie_filling', u'_prosciutto', u'_real_mayonnaise', u'_refried_beans', u'_refrigerated_crescent_dinner_rolls', u'_rins_drain', u'_salmon', u'_sharp_ched

In [8]:
X_train = vectorizer.transform(recipes).toarray()
Y_train = labels

In [9]:
from sklearn.naive_bayes import BernoulliNB
clf = BernoulliNB()
clf.fit(X_train, Y_train)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [10]:
input_recipe = raw_input("What is your recipe? \n> ")
while input_recipe != 'quit':
    input_recipe = input_recipe.lower()
    X_input = vectorizer.transform([input_recipe]).toarray()
    prediction = clf.predict(X_input)[0]
    
    print('\nIdentified Ingredients: \n> %s'%vectorizer.inverse_transform(X_input))
    
    print('\nPredicted Cuisine Type: \n> %s'%prediction)
    print('\n====================================')
    
    input_recipe = input("What is your recipe? \n> ")

What is your recipe? 
> quit


NameError: name 'quit' is not defined

In [11]:
from sklearn.pipeline import Pipeline
NB_model = Pipeline([('vectorizer', vectorizer),('NB', clf)])

In [14]:
import pickle
filename = 'NB_model_v1.sav'
pickle.dump(NB_model, open(filename,'wb'))

In [13]:
NB_model.predict(list(['hi', 'hi']))

array([u'italian', u'italian'], dtype='<U12')